In [1]:
import numpy as np
import scipy as sp
import bordado as bd
import matplotlib.pyplot as plt
import ipywidgets

In [2]:
def defasagem_ps(epicentro, estações, vp, vs):
    """
    Calcula a diferença de tempo entre a chegada das ondas P e S.
    """
    x, y = epicentro
    xs, ys = estações
    distancia = np.sqrt((x - xs)**2 + (y - ys)**2)
    return distancia * (1 / vs - 1 / vp)

In [3]:
def mapa_objetivo(região, espaçamento, estações, vp, vs, dados, ax=None):
    """
    Plota a função objetivo com linhas de contorno coloridas.
    """
    epicentros = bd.grid_coordinates(região, spacing=espaçamento)
    xs, ys = estações
    predito = defasagem_ps(
        epicentros, 
        (xs[:, np.newaxis, np.newaxis], ys[:, np.newaxis, np.newaxis]), 
        vp, vs,
    )
    objetivo = np.linalg.norm(dados[:, np.newaxis, np.newaxis] - predito, axis=0)**2
    if ax is None:
        ax = plt.gca()
    ax.contourf(*epicentros, objetivo, 80, cmap="YlGn_r")

In [4]:
def jacobiana(epicentro, estações, vp, vs):
    """
    Calcula a matriz Jacobiana.
    """
    x, y = epicentro
    xs, ys = estações
    A = np.empty((xs.size, 2))
    distancia = np.sqrt((x - xs)**2 + (y - ys)**2)
    A[:, 0] = (x - xs) / distancia * (1 / vs - 1 / vp)
    A[:, 1] = (y - ys) / distancia * (1 / vs - 1 / vp)
    return A

In [5]:
def estima_epicentro_newton(inicial, estações, vp, vs, dados, max_it):
    """
    Estima o epicentro com método de Newton.
    """
    epicentro = np.array(inicial)
    residuos = dados - defasagem_ps(epicentro, estações, vp, vs)
    objetivo = [np.linalg.norm(residuos)**2]
    passos = [epicentro]
    for i in range(max_it):
        A = jacobiana(epicentro, estações, vp, vs)
        passo = sp.linalg.solve(A.T @ A, A.T @ residuos)
        epicentro = epicentro + passo
        residuos = dados - defasagem_ps(epicentro, estações, vp, vs)
        objetivo.append(np.linalg.norm(residuos)**2)
        passos.append(epicentro)
    return np.transpose(passos), objetivo        

def estima_epicentro_levmarq(inicial, estações, vp, vs, dados, max_it, alpha, dalpha):
    """
    Estima o epicentro com método de Levemberg-Marquardt.
    """
    epicentro = np.array(inicial)
    residuos = dados - defasagem_ps(epicentro, estações, vp, vs)
    objetivo = [np.linalg.norm(residuos)**2]
    passos = [epicentro]
    I = np.eye(2)
    for i in range(max_it):
        A = jacobiana(epicentro, estações, vp, vs)
        hessiana = A.T @ A
        precondicionador = np.diag(1 / np.abs(np.diagonal(hessiana)))
        hessiana = precondicionador @ hessiana    
        gradiente = -precondicionador @ A.T @ residuos
        for j in range(100):             
            passo = sp.linalg.solve(hessiana + alpha * I, -gradiente)            
            residuos = dados - defasagem_ps(epicentro + passo, estações, vp, vs)
            nova_objetivo = np.linalg.norm(residuos)**2
            if nova_objetivo > objetivo[-1]:
                alpha *= dalpha
            else:
                alpha /= dalpha
                break
        epicentro = epicentro + passo
        residuos = dados - defasagem_ps(epicentro, estações, vp, vs)
        objetivo.append(np.linalg.norm(residuos)**2)
        passos.append(epicentro)
    return np.transpose(passos), objetivo           

In [6]:
max_it = 20
max_estações = 10

def plota_iterações(n_estações, iterações, ruído, alpha, semente_estações, semente_inicial):
    """
    Faz as inversões e plota as iterações em um mapa da função objetivo.
    """
    região = (0, 600e3, 0, 500e3)
    epicentro_verdadeiro = 320e3, 260e3
    vp, vs = 4000, 2000
    região_pad = bd.pad_region(região, -80e3)
    estações = tuple(
        i[:n_estações] for i in bd.random_coordinates(
            região_pad, size=max_estações, random_seed=semente_estações,
        )
    )
    dados = defasagem_ps(epicentro_verdadeiro, estações, vp, vs)
    dados += np.random.default_rng(42).normal(0, ruído / 100 * np.abs(dados).max(), dados.shape)
    inicial = np.array(bd.random_coordinates(
        região, size=1, random_seed=semente_inicial,
    )).ravel()
    passos_newton, objetivo_newton = estima_epicentro_newton(
        inicial, estações, vp, vs, dados, max_it
    )
    passos_levmarq, objetivo_levmarq = estima_epicentro_levmarq(
        inicial, estações, vp, vs, dados, max_it, alpha, 10,
    )
    fig = plt.figure(figsize=(7, 7), layout="constrained")
    figures = fig.subfigures(2, 1, height_ratios=[2, 1])    
    ax = figures[0].subplots(1, 1)    
    mapa_objetivo(região, 5e3, estações, vp, vs, dados, ax)
    ax.plot(*passos_newton[:, :iterações + 1], ".-", linewidth=2.5, markersize=8)    
    ax.plot(*passos_levmarq[:, :iterações + 1], ".--", linewidth=2.5, markersize=8)
    ax.plot(*inicial, "sy", markersize=7)
    ax.plot(*estações, "^k", markersize=7)
    ax.plot(*epicentro_verdadeiro, "*r", markersize=8)
    ax.set_xlim(*região[:2])
    ax.set_ylim(*região[2:])
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")  
    ax = figures[1].subplots(1, 1)    
    ax.plot(objetivo_newton[:iterações + 1], ".-", linewidth=2.5, markersize=8)
    ax.plot(objetivo_levmarq[:iterações + 1], ".-", linewidth=2.5, markersize=8)
    ax.set_xlabel("Iterações")
    ax.set_ylabel("Função objetivo (s²)")
    ax.set_xticks(range(0, max_it + 1, 1))
    ax.set_xlim(0, max_it + 1)
    ax.set_ylim(
        0.9 * np.min((objetivo_levmarq, objetivo_newton)),
        1.1 * np.max((objetivo_levmarq, objetivo_newton)),
    )
    ax.grid(axis="y")

In [7]:
def aplicativo():
    n_estações = ipywidgets.IntSlider(value=3, min=2, max=max_estações, step=1, continuous_update=False)
    iterações = ipywidgets.IntSlider(value=0, min=0, max=max_it, step=1, continuous_update=False)
    ruído = ipywidgets.IntSlider(value=0, min=0, max=30, step=1, continuous_update=False)
    alpha = ipywidgets.FloatLogSlider(value=1, min=-2, max=2, step=0.5, continuous_update=False)
    semente_estações = ipywidgets.IntSlider(value=17, min=0, max=50, step=1, continuous_update=False)
    semente_inicial = ipywidgets.IntSlider(value=9, min=0, max=50, step=1, continuous_update=False)
    output = ipywidgets.interactive_output(
        plota_iterações, 
        dict(
            n_estações=n_estações, iterações=iterações, alpha=alpha, ruído=ruído,
            semente_estações=semente_estações, semente_inicial=semente_inicial,
        )
    )
    app = ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.HTML("<h3><b>Controles:</b></h3>"),
            ipywidgets.Label("Número de estações"), n_estações, 
            ipywidgets.Label("Iterações"), iterações, 
            ipywidgets.Label("Ruído nos dados (%)"), ruído, 
            ipywidgets.Label("Regularização do passo ($\\alpha$)"), alpha, 
            ipywidgets.Label("Semente das estações"), semente_estações,
            ipywidgets.Label("Semente do chute inicial"), semente_inicial,
        ]),
        output
    ])
    return app

In [8]:
aplicativo()